In [1]:
!pip install pynput
!pip install  torch_lr_finder
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 11.4 MB/s eta 0:00:00
  Created wheel for evdev: filename=evdev-1.6.1-cp310-cp310-linux_x86_64.whl size=45847 sha256=6883bc59e0968a3bb93740bdb98d0c9c74edc5312d8017f08380747b03ec49d4
  Stored in directory: /root/.cache/pip/wheels/bb/6e/51/0c0f5677bd58c532c42e13e7d93803fbf8d5d3bf5bfc5b0220
Successfully built evdev


In [2]:
# !mkdir store

In [3]:
!ls store

'epoch=29-step=15027.ckpt'     'epoch=36-step=19166.ckpt'
'epoch=31-step=15037.ckpt'     'epoch=37-step=19684.ckpt'
'epoch=32-step=15042.ckpt'     'epoch=38-step=20202.ckpt'
'epoch=35-step=18648.ckpt'     'epoch=4-step=25.ckpt'
'epoch=36-step=19166-v1.ckpt'


In [28]:
!rm -r /kaggle/working/yolov3_object_detection/
%cd /kaggle/working
!git clone https://github.com/venkyyuvy/yolov3_object_detection.git
%cd /kaggle/working/yolov3_object_detection
!git checkout modified_callback
# !python main.py

/kaggle/working
Cloning into 'yolov3_object_detection'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 168 (delta 107), reused 116 (delta 58), pack-reused 0
Receiving objects: 100% (168/168), 38.87 KiB | 1.55 MiB/s, done.
Resolving deltas: 100% (107/107), done.
/kaggle/working/yolov3_object_detection
Branch 'modified_callback' set up to track remote branch 'modified_callback' from 'origin'.
Switched to a new branch 'modified_callback'


In [29]:
from model import YOLOv3
import torch
fname = '../store/epoch=38-step=20202.ckpt'
checkpoint = torch.load(fname, map_location=torch.device('cuda'))
model_state_dict = checkpoint['state_dict']
model = YOLOv3(num_classes=20)
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [30]:
import config
import utils
from dataset import YOLODataModule

yolo_datamodule = YOLODataModule(
        train_csv_path=config.DATASET + "/train.csv",
        test_csv_path=config.DATASET + "/test.csv",
    )

yolo_datamodule.setup()

In [34]:
model.eval()
model = model.to(config.DEVICE)
pred_boxes, true_boxes = utils.get_evaluation_bboxes(
        loader=yolo_datamodule.test_dataloader(),
        model=model,
        iou_threshold=config.NMS_IOU_THRESH,
        anchors=config.ANCHORS,
        threshold=config.CONF_THRESHOLD,
    )

100%|██████████| 310/310 [22:14<00:00,  4.31s/it]


In [36]:
from utils import mean_average_precision
map_val = mean_average_precision(
    pred_boxes=pred_boxes,
    true_boxes=true_boxes,
    iou_threshold=config.MAP_IOU_THRESH,
    box_format="midpoint",
    num_classes=config.NUM_CLASSES,
).item()
print("+++ MAP: ", map_val)


100%|██████████| 20/20 [03:44<00:00, 11.24s/it]

+++ MAP:  0.570273756980896
